## Imports

In [130]:
import requests
import pandas as pd
import numpy as np
import pyodbc 
import import_ipynb
import functions as f

## Azure SQL Server connection configuration

In [96]:
server = 'lirkovsrv.database.windows.net'
database = 'lirkovdb'
username = 'lirkov'
password = '{LLirkoff31}'   
driver= '{ODBC Driver 17 for SQL Server}'

## Extract

In [97]:
#f.downloadFile('./Raw/WHO-COVID19-CASES.csv','https://covid19.who.int/WHO-COVID-19-global-data.csv')

## Transform

In [117]:
# create DataFrame

df_cases = pd.read_csv('./Raw/WHO-COVID19-CASES.csv')

In [118]:
# clean column names

df_cases.columns = f.fixColumnNames(df_cases)

In [119]:
# change data types

f.changeDataType(df_cases, 'DateReported', 'datetime64')

In [120]:
# create DataFrame for the 'countries' dataset. It will be used to bring country iso code to the main DataFrame

df_countries = pd.read_csv('./Processed/COUNTRIES_DETAILED.csv')

In [121]:
# left join df_countries to df_cases to bring the 3-letter country code

df_merged = df_cases.merge(df_countries[['IsoCode', 'Alpha2']], left_on='CountryCode', right_on='Alpha2', how='left').drop(columns=['Alpha2', 'CountryCode'])

In [122]:
# drop rows with missing 'IsoCode'

df_merged = df_merged.loc[df_merged['IsoCode'].isna() == False]

In [135]:
# reorder columns

df_merged = f.reindexCols(df_merged, [0,1,7,2,3,4,5,6])

In [138]:
# create 'Key' column by concatenating 'CountryCode' and 'DateReported'

df_merged['Key'] = df_merged['IsoCode'] + df_merged['DateReported'].astype(str).str.replace('-','')

In [139]:
# set 'Key' as an index and ensure that there are no duplicates

df_merged.set_index('Key', inplace=True, verify_integrity=True)

## Load

In [141]:
# save as parquet to preserve data types

df_merged.to_parquet('./Processed/WHO-COVID19-CASES.parquet')

In [47]:
# conn =  pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
# cursor = conn.cursor()



# for index, row in df.iterrows():
#      cursor.execute("INSERT INTO dbo.WHO_Covid19 (DateReported, CountryCode, WhoRegion, NewCases, CumulativeCases, NewDeaths, CumulativeDeaths) values(?,?,?,?,?,?,?)", row.DateReported, row.CountryCode, row.WhoRegion, row.NewCases, row.CumulativeCases, row.NewDeaths, row.CumulativeDeaths)
# conn.commit()
# cursor.close()